# DataScraping Notebook

### Import Relevent Modules for DataScraping

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
import time
from fake_useragent import UserAgent

In [ ]:
Webscraping Functio

In [2]:
def scrape_and_export_descriptions(input_csv, output_csv):
    # Load the input CSV
    data = pd.read_csv(input_csv)
   
    # Rename the column
    data = data.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)': 'URL'})
   
    # Function to expand and extract text
    def expand_and_extract_text(url):
        
        chrome_options = Options()
        #Intialize
        driver = webdriver.Chrome(options=chrome_options)

        try:
            # Navigate to the URL
            driver.get(url)
           
            # Print the URL being processed
            print(f"Processing URL: {url}")

            # Scroll down by 600 pixels
            driver.execute_script("window.scrollBy(0, 600);")

            # Click the Show More Button to Expand Description
            button = driver.find_element(By.XPATH, '//*[@id="marketingRemarks-preview"]/div[2]/div/button')

            # Click the button
            button.click()

            # Wait for .5 seconds to allow the content to expand 
            time.sleep(.5)

            # Get the page source
            page_source = driver.page_source

            # Parse the page with BeautifulSoup
            soup = BeautifulSoup(page_source, 'html.parser')

            # Extract the text content using BeautifulSoup
            text_content = soup.find('div', {'class': 'house-info', 'id': 'house-info', 'data-rf-test-id': 'house-info'}).get_text()

            return text_content

        except Exception as e:
            print(f"Error processing URL: {url}")
            print(e)

        finally:
            # Close the web driver
            driver.quit()

  
    data['Description'] = data['URL'].apply(expand_and_extract_text)


    data.to_csv(output_csv, index=False)